# TP 2 : SPARK Preprocessing

### Kaelig Castor

- Charger un fichier csv dans un dataFrame
- Pre-processing: cleaning, filters, feature engineering => filter, select, drop, na.fill, join, udf, distinct, count, describe, collect
- Sauver le dataframe au format parquet

In [84]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.types.IntegerType
// Des réglages optionnels du job spark. Les réglages par défaut fonctionnent très bien pour ce TP.
// On vous donne un exemple de setting quand même
val conf = new SparkConf().setAll(Map(
      "spark.scheduler.mode" -> "FIFO",
      "spark.speculation" -> "false",
      "spark.reducer.maxSizeInFlight" -> "48m",
      "spark.serializer" -> "org.apache.spark.serializer.KryoSerializer",
      "spark.kryoserializer.buffer.max" -> "1g",
      "spark.shuffle.file.buffer" -> "32k",
      "spark.default.parallelism" -> "12",
      "spark.sql.shuffle.partitions" -> "12"
    ))

// Initialisation du SparkSession qui est le point d'entrée vers Spark SQL (donne accès aux dataframes, aux RDD,
// création de tables temporaires, etc., et donc aux mécanismes de distribution des calculs)
val spark = SparkSession
      .builder
      .config(conf)
      .appName("TP Spark : Preprocessor")
      .getOrCreate()

    /*******************************************************************************
      *
      *       TP 2
      *
      *       - Charger un fichier csv dans un dataFrame
      *       - Pre-processing: cleaning, filters, feature engineering => filter, select, drop, na.fill, join, udf, distinct, count, describe, collect
      *       - Sauver le dataframe au format parquet
      *
      *       if problems with unimported modules => sbt plugins update
      *
      ********************************************************************************/

println("\n")
println("Hello World ! from Preprocessor")
println("\n")
val df: DataFrame = spark
        .read
        .option("header", true)        // utilise la première ligne du (des) fichier(s) comme
        .option("inferSchema", "true") // pour inférer le type de chaque colonne (Int,
        .option("escape", "\"")
        .csv("/mnt/d/09_SPARK/cours-spark-telecom-master/data/train_clean.csv")
println(s"Nombre de lignes : ${df.count}")
println(s"Nombre de colonnes : ${df.columns.length}")
df.show()
df.printSchema()
val dfCasted: DataFrame = df
      .withColumn("goal", $"goal".cast("Int"))
      .withColumn("deadline" , $"deadline".cast("Int"))
      .withColumn("state_changed_at", $"state_changed_at".cast("Int"))
      .withColumn("created_at", $"created_at".cast("Int"))
      .withColumn("launched_at", $"launched_at".cast("Int"))
      .withColumn("backers_count", $"backers_count".cast("Int"))
      .withColumn("final_status", $"final_status".cast("Int"))

dfCasted.printSchema()



Hello World ! from Preprocessor


Nombre de lignes : 108129
Nombre de colonnes : 14
+--------------+--------------------+--------------------+-------+--------------------+---------------------+-------+--------+----------+----------------+----------+-----------+-------------+------------+
|    project_id|                name|                desc|   goal|            keywords|disable_communication|country|currency|  deadline|state_changed_at|created_at|launched_at|backers_count|final_status|
+--------------+--------------------+--------------------+-------+--------------------+---------------------+-------+--------+----------+----------------+----------+-----------+-------------+------------+
|kkst1451568084| drawing for dollars|I like drawing pi...|   20.0| drawing-for-dollars|                false|     US|     USD|1241333999|      1241334017|1240600507| 1240602723|            3|           1|
|kkst1474482071|Sponsor Dereck Bl...|I  Dereck Blackbu...|  300.0|sponsor-dereck-bl...|       

import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.types.IntegerType
conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@57c0a744
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7e78e505
df: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]
dfCasted: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]


In [85]:
dfCasted
      .select("goal", "backers_count", "final_status")
      .describe()
      .show

+-------+-----------------+------------------+-------------------+
|summary|             goal|     backers_count|       final_status|
+-------+-----------------+------------------+-------------------+
|  count|           108129|            108129|             108129|
|   mean|36726.22826438791|123.51666065532835| 0.3196274819891056|
| stddev|971902.7051687702|1176.7451621583891|0.46633439282834627|
|    min|                0|                 0|                  0|
|    max|        100000000|            219382|                  1|
+-------+-----------------+------------------+-------------------+



In [86]:
dfCasted.groupBy("disable_communication").count.orderBy($"count".desc).show(100)

+---------------------+------+
|disable_communication| count|
+---------------------+------+
|                false|107806|
|                 true|   323|
+---------------------+------+



In [87]:
dfCasted.groupBy("country").count.orderBy($"count".desc).show(100)

+-------+-----+
|country|count|
+-------+-----+
|     US|92033|
|     GB| 8758|
|     CA| 3736|
|     AU| 1880|
|     NL|  705|
|     NZ|  355|
|     SE|  240|
|     DK|  196|
|     NO|  114|
|     IE|  111|
|     DE|    1|
+-------+-----+



In [88]:
dfCasted.groupBy("currency").count.orderBy($"count".desc).show(100)

+--------+-----+
|currency|count|
+--------+-----+
|     USD|92033|
|     GBP| 8758|
|     CAD| 3736|
|     AUD| 1880|
|     EUR|  817|
|     NZD|  355|
|     SEK|  240|
|     DKK|  196|
|     NOK|  114|
+--------+-----+



In [89]:
dfCasted.select("deadline").dropDuplicates.show()

+----------+
|  deadline|
+----------+
|1244264400|
|1245026160|
|1247607960|
|1248486180|
|1249077600|
|1249932360|
|1249962720|
|1251772980|
|1251896400|
|1251953940|
|1253150160|
|1253595600|
|1254412800|
|1254628740|
|1254767400|
|1254851580|
|1255027560|
|1255060560|
|1255402080|
|1255622400|
+----------+
only showing top 20 rows



In [90]:
dfCasted.groupBy("state_changed_at").count.orderBy($"count".desc).show(100)

+----------------+-----+
|state_changed_at|count|
+----------------+-----+
|      1414814340|   26|
|      1420088342|   20|
|      1425185942|   17|
|      1427860741|   14|
|      1401483613|   14|
|      1414800010|   12|
|      1401595142|   12|
|      1414817940|   11|
|      1409543941|   11|
|      1409544011|   10|
|      1343793542|   10|
|      1372651141|   10|
|      1417496340|   10|
|      1430452741|   10|
|      1359694743|   10|
|      1404197943|    9|
|      1364788742|    9|
|      1343879941|    9|
|      1425078013|    9|
|      1346471944|    9|
|      1349074740|    9|
|      1423976340|    9|
|      1383289142|    9|
|      1412135943|    8|
|      1420091941|    8|
|      1383278341|    8|
|      1412136011|    8|
|      1414900740|    7|
|      1388552340|    7|
|      1430452742|    7|
|      1351742342|    7|
|      1430539144|    7|
|      1362113941|    7|
|      1412146741|    7|
|      1425272344|    7|
|      1296536343|    7|
|      1370059144|    7|


In [91]:
dfCasted.groupBy("backers_count").count.orderBy($"count".desc).show(100)

+-------------+-----+
|backers_count|count|
+-------------+-----+
|            0|12832|
|            1| 8819|
|            2| 5978|
|            3| 4224|
|            4| 3185|
|            5| 2684|
|            6| 2254|
|            7| 2012|
|            8| 1699|
|            9| 1462|
|           10| 1419|
|           11| 1270|
|           13| 1227|
|           12| 1195|
|           14| 1184|
|           15| 1090|
|           18| 1052|
|           16| 1047|
|           17| 1011|
|           19|  988|
|           20|  903|
|           21|  879|
|           23|  836|
|           22|  836|
|           24|  815|
|           25|  790|
|           26|  789|
|           27|  745|
|           28|  717|
|           29|  681|
|           34|  671|
|           30|  658|
|           32|  639|
|           33|  614|
|           38|  594|
|           31|  593|
|           35|  583|
|           36|  575|
|           40|  574|
|           37|  565|
|           39|  554|
|           41|  535|
|         

In [92]:
dfCasted.select("goal", "final_status").show(30)

+-----+------------+
| goal|final_status|
+-----+------------+
|   20|           1|
|  300|           0|
|   30|           0|
|  500|           1|
| 2000|           0|
|  700|           0|
|  250|           0|
| 1000|           1|
| 5000|           0|
| 3500|           0|
|30000|           0|
|  300|           0|
| 1500|           1|
|  500|           1|
|  500|           0|
| 1000|           1|
|  600|           0|
| 1500|           1|
| 3500|           0|
| 1000|           1|
|  365|           1|
|  500|           1|
|  400|           1|
|  100|           1|
|  250|           1|
| 3000|           1|
|  640|           0|
| 3500|           1|
|  300|           1|
| 1000|           1|
+-----+------------+
only showing top 30 rows



In [93]:
dfCasted.groupBy("country", "currency").count.orderBy($"count".desc).show(50)

+-------+--------+-----+
|country|currency|count|
+-------+--------+-----+
|     US|     USD|92033|
|     GB|     GBP| 8758|
|     CA|     CAD| 3736|
|     AU|     AUD| 1880|
|     NL|     EUR|  705|
|     NZ|     NZD|  355|
|     SE|     SEK|  240|
|     DK|     DKK|  196|
|     NO|     NOK|  114|
|     IE|     EUR|  111|
|     DE|     EUR|    1|
+-------+--------+-----+



On enlève la colonne "disable_communication" qui est très largement
majoritairement à false, il n'y a que 322 true (négligeable)

In [94]:
val df2: DataFrame = dfCasted.drop("disable_communication")

df2: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 11 more fields]


In [95]:
dfCasted
    .select("disable_communication", "backers_count", "state_changed_at")
    .show
print(dfCasted.columns.toList)

+---------------------+-------------+----------------+
|disable_communication|backers_count|state_changed_at|
+---------------------+-------------+----------------+
|                false|            3|      1241334017|
|                false|            2|      1242432018|
|                false|            0|      1243027818|
|                false|           18|      1243556121|
|                false|            1|      1243770317|
|                false|           14|      1243816219|
|                false|            2|      1243872028|
|                false|           32|      1244088022|
|                false|           44|      1244264422|
|                false|           18|      1244946632|
|                false|            7|      1245026721|
|                false|            2|      1245038428|
|                false|           28|      1245042919|
|                false|           98|      1245092431|
|                false|            3|      1245528920|
|         

In [96]:
df2
    .select("backers_count", "state_changed_at")
    .describe()
    .show

+-------+------------------+--------------------+
|summary|     backers_count|    state_changed_at|
+-------+------------------+--------------------+
|  count|            108129|              108129|
|   mean|123.51666065532835|1.3801529957698119E9|
| stddev|1176.7451621583891|4.2664018444684386E7|
|    min|                 0|          1241334017|
|    max|            219382|          1433096940|
+-------+------------------+--------------------+



#### Pour enlever les données du futur, on retire les colonnes "backers_count" et "state_changed_at".

In [97]:
val dfNoFutur: DataFrame = df2.drop("backers_count", "state_changed_at") 

dfNoFutur: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]


In [98]:
df2
    .select("backers_count", "state_changed_at")
    .describe()
    .show
print(df2.columns.toList)

+-------+------------------+--------------------+
|summary|     backers_count|    state_changed_at|
+-------+------------------+--------------------+
|  count|            108129|              108129|
|   mean|123.51666065532835|1.3801529957698119E9|
| stddev|1176.7451621583891|4.2664018444684386E7|
|    min|                 0|          1241334017|
|    max|            219382|          1433096940|
+-------+------------------+--------------------+

List(project_id, name, desc, goal, keywords, country, currency, deadline, state_changed_at, created_at, launched_at, backers_count, final_status)

In [103]:
print(dfNoFutur.columns.toList)

List(country, currency)

Lorsque country = "False" le country à l'air d'être dans
currency.

In [100]:
dfNoFutur.filter($"country" === "False")
    .groupBy("currency")
    .count
    .orderBy($"count".desc)
    .show(50)

+--------+-----+
|currency|count|
+--------+-----+
+--------+-----+



Creation de deux UDF.

- cleanCountry : si country = "False" prendre la valeur de currency, sinon si country est une chaîne de caractères de taille autre que 2 remplacer par null, et sinon laisser la valeur country actuelle. On veut les résultat dans une nouvelle colonne country2.

In [65]:
def cleanCountry(country: String, currency: String): String = {
    if (country == "False")
        currency 
    else if (country.length != 2 )
        null
    else
        country
}

cleanCountry: (country: String, currency: String)String


- cleanCurrency : si currency.length != 3 currency prend la valeur null, sinon laisser
la valeur currency actuelle. On veut les résultats dans une nouvelle colonne
currency2.

In [66]:
def cleanCurrency(currency: String): String = {
    if (currency != null && currency.length != 3)
        null
    else
        currency
}

cleanCurrency: (currency: String)String


In [67]:
val cleanCountryUdf = udf(cleanCountry _)
val cleanCurrencyUdf = udf(cleanCurrency _)
val dfCountry_2: DataFrame = dfNoFutur
    .withColumn("country2", cleanCountryUdf($"country", $"currency"))
    .withColumn("currency2", cleanCurrencyUdf($"currency"))
    .drop("country", "currency")

cleanCountryUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,StringType,Some(List(StringType, StringType)))
cleanCurrencyUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
dfCountry_2: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]


In [68]:
print(dfCountry_2.columns.toList)
dfCountry_2
    .select("country2", "currency2")
    .describe()
    .show

List(project_id, name, desc, goal, keywords, deadline, created_at, launched_at, final_status, country2, currency2)+-------+--------+---------+
|summary|country2|currency2|
+-------+--------+---------+
|  count|  108129|   108129|
|   mean|    null|     null|
| stddev|    null|     null|
|    min|      AU|      AUD|
|    max|      US|      USD|
+-------+--------+---------+



#### ou encore, en utilisant sql.functions.when:

In [69]:
import org.apache.spark.sql.functions.{when, _}
val dfCountry: DataFrame = dfNoFutur
      .withColumn("country2", when($"country" === "False", $"currency").otherwise($"country"))
      .withColumn("currency2", when($"country".isNotNull && length($"currency") =!= 3, null).otherwise($"currency"))
      .drop("country", "currency")

import org.apache.spark.sql.functions.{when, _}
dfCountry: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]


In [70]:
dfNoFutur
    .select("country", "currency")
    .describe()
    .show

+-------+-------+--------+
|summary|country|currency|
+-------+-------+--------+
|  count| 108129|  108129|
|   mean|   null|    null|
| stddev|   null|    null|
|    min|     AU|     AUD|
|    max|     US|     USD|
+-------+-------+--------+



In [71]:
dfCountry
    .select("country2", "currency2")
    .describe()
    .show

+-------+--------+---------+
|summary|country2|currency2|
+-------+--------+---------+
|  count|  108129|   108129|
|   mean|    null|     null|
| stddev|    null|     null|
|    min|      AU|      AUD|
|    max|      US|      USD|
+-------+--------+---------+



In [72]:
dfCountry
      .select("deadline", "launched_at")
      .describe()
      .show

+-------+--------------------+--------------------+
|summary|            deadline|         launched_at|
+-------+--------------------+--------------------+
|  count|              108129|              108129|
|   mean|1.3802484980048554E9|1.3772990047093103E9|
| stddev| 4.270222122091535E7| 4.294421262600255E7|
|    min|          1241333999|          1240602723|
|    max|          1433096938|          1432658473|
+-------+--------------------+--------------------+



In [73]:
print(df.filter($"country".isNotNull))

[project_id: string, name: string ... 12 more fields]

In [74]:
df2.withColumn("country", lower($"country")) 

res59: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 11 more fields]


#### On ajoute une colonne days_campaign qui représente la durée de la campagne en jours (le nombre de jours entre launched_at et deadline).

In [75]:
val dfCountry2 = dfCountry
    .withColumn("days_campaign", round( ( dfCountry.col("deadline")-dfCountry.col("launched_at") ) /86400 *1000)/1000 )
dfCountry2
    .select("deadline", "launched_at", "days_campaign")
    .describe()
    .show
//dfCountry.apply('days_campaign')
//    .withColumn("days_campaign", dateddiff(dfCountry.col("deadline"), dfCountry.col("launched_at")/365))


+-------+--------------------+--------------------+------------------+
|summary|            deadline|         launched_at|     days_campaign|
+-------+--------------------+--------------------+------------------+
|  count|              108129|              108129|            108129|
|   mean|1.3802484980048554E9|1.3772990047093103E9|34.137648965587374|
| stddev| 4.270222122091535E7| 4.294421262600255E7| 12.96770021607119|
|    min|          1241333999|          1240602723|               1.0|
|    max|          1433096938|          1432658473|            91.958|
+-------+--------------------+--------------------+------------------+



dfCountry2: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 10 more fields]


In [76]:
dfCountry2
    .select("days_campaign")
    .show

+-------------+
|days_campaign|
+-------------+
|        8.464|
|       16.822|
|         9.99|
|       29.965|
|       29.969|
|       27.212|
|       24.719|
|       30.298|
|       32.216|
|       30.938|
|       34.376|
|       29.046|
|       30.938|
|       46.909|
|       36.977|
|        33.18|
|       13.197|
|       61.903|
|       23.528|
|       32.461|
+-------------+
only showing top 20 rows



#### On ajoute une colonne hours_prepa qui représente le nombre d’heures de préparation de la campagne entre created_at et launched_at. On arrondit le résultat à 3 chiffres après la virgule.

In [77]:
val dfCountry3 = dfCountry2
    .withColumn("hours_prepa", round((dfCountry2.col("launched_at")-dfCountry2.col("created_at"))/3600*1000)/1000)
dfCountry2
    .select("deadline", "launched_at", "days_campaign")
    .describe()
    .show

+-------+--------------------+--------------------+------------------+
|summary|            deadline|         launched_at|     days_campaign|
+-------+--------------------+--------------------+------------------+
|  count|              108129|              108129|            108129|
|   mean|1.3802484980048554E9|1.3772990047093103E9|34.137648965587374|
| stddev| 4.270222122091535E7| 4.294421262600255E7| 12.96770021607119|
|    min|          1241333999|          1240602723|               1.0|
|    max|          1433096938|          1432658473|            91.958|
+-------+--------------------+--------------------+------------------+



dfCountry3: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 11 more fields]


In [78]:
dfCountry3
    .select("created_at","deadline","launched_at","days_campaign","hours_prepa")
    .show

+----------+----------+-----------+-------------+-----------+
|created_at|  deadline|launched_at|days_campaign|hours_prepa|
+----------+----------+-----------+-------------+-----------+
|1240600507|1241333999| 1240602723|        8.464|      0.616|
|1240960224|1242429000| 1240975592|       16.822|      4.269|
|1242163613|1243027560| 1242164398|         9.99|      0.218|
|1240963795|1243555740| 1240966730|       29.965|      0.815|
|1241177914|1243769880| 1241180541|       29.969|       0.73|
|1241050799|1243815600| 1241464468|       27.212|    114.908|
|1241725172|1243872000| 1241736308|       24.719|      3.093|
|1241460541|1244088000| 1241470291|       30.298|      2.708|
|1241415164|1244264400| 1241480901|       32.216|      18.26|
|1242268157|1244946540| 1242273460|       30.938|      1.473|
|1241829376|1245026160| 1242056094|       34.376|     62.977|
|1242523061|1245038400| 1242528805|       29.046|      1.596|
|1242364202|1245042600| 1242369560|       30.938|      1.488|
|1241034

#### On supprime les colonnes launched_at, created_at, et deadline, elles ne sont pas exploitables pour un modèle.

In [79]:
val dfCountry4: DataFrame = dfCountry3
      .drop("launched_at", "created_at", "deadline")
print(dfCountry4.columns.toList)
dfCountry4.show
dfCountry4.printSchema()

List(project_id, name, desc, goal, keywords, final_status, country2, currency2, days_campaign, hours_prepa)+--------------+--------------------+--------------------+-----+--------------------+------------+--------+---------+-------------+-----------+
|    project_id|                name|                desc| goal|            keywords|final_status|country2|currency2|days_campaign|hours_prepa|
+--------------+--------------------+--------------------+-----+--------------------+------------+--------+---------+-------------+-----------+
|kkst1451568084| drawing for dollars|I like drawing pi...|   20| drawing-for-dollars|           1|      US|      USD|        8.464|      0.616|
|kkst1474482071|Sponsor Dereck Bl...|I  Dereck Blackbu...|  300|sponsor-dereck-bl...|           0|      US|      USD|       16.822|      4.269|
| kkst183622197|       Mr. Squiggles|So I saw darkpony...|   30|        mr-squiggles|           0|      US|      USD|         9.99|      0.218|
| kkst597742710|Help me write

dfCountry4: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 8 more fields]


#### On met les colonnes name, desc, et keywords en minuscules, et on ajoute une colonne text, qui contient la concaténation des Strings des colonnes name, desc, et keywords, avec un espace entre les chaînes de caractères concaténées : 

In [80]:
import org.apache.spark.sql.functions.lower
val dfCountry5: DataFrame = dfCountry4
    .withColumn("name", lower($"name")) 
    .withColumn("desc", lower($"desc"))
    .withColumn("keywords", lower($"keywords"))
dfCountry5.select("name","desc","keywords").show()
print(dfCountry5.columns.toList)

+--------------------+--------------------+--------------------+
|                name|                desc|            keywords|
+--------------------+--------------------+--------------------+
| drawing for dollars|i like drawing pi...| drawing-for-dollars|
|sponsor dereck bl...|i  dereck blackbu...|sponsor-dereck-bl...|
|       mr. squiggles|so i saw darkpony...|        mr-squiggles|
|help me write my ...|do your part to h...|help-me-write-my-...|
|support casting m...|i m nearing compl...|support-casting-m...|
|        daily digest|i m a fledgling v...|        daily-digest|
|igoozex - free ip...|i am an independe...|igoozex-free-ipho...|
|drive a faster ca...|drive a faster ca...|drive-a-faster-ca...|
|"""""""""""""""""...|opening friday  j...|lostles-at-tinys-...|
|choose your own a...|this project is f...|choose-your-own-a...|
|anatomy of a cred...|i am an independe...|anatomy-of-a-cred...|
|no-bit: an artist...|i want to create ...|no-bit-an-artist-...|
|indie nerd board ...|pic

import org.apache.spark.sql.functions.lower
dfCountry5: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 8 more fields]


In [117]:
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions.concat_ws
//val dfCountry6: DataFrame = dfCountry5
//    .withColumn("text",concat_ws(" ", "name", "desc", "keywords"))
val colsToConcat = Seq(col("name"), col("desc"), col("keywords"))
//dftext=dfCountry5.select(concat_ws(" ", colsToConcat :_*)).show
val dfCountry6: DataFrame = dfCountry5
    .withColumn("text",concat_ws(" ", colsToConcat :_*))
dfCountry6.select("text").show()
print(dfCountry6.columns.toList)

+--------------------+
|                text|
+--------------------+
|drawing for dolla...|
|sponsor dereck bl...|
|mr. squiggles so ...|
|help me write my ...|
|support casting m...|
|daily digest i m ...|
|igoozex - free ip...|
|drive a faster ca...|
|"""""""""""""""""...|
|choose your own a...|
|anatomy of a cred...|
|no-bit: an artist...|
|indie nerd board ...|
|icons for your ip...|
|happy valley: dex...|
|project pedal pro...|
|frank magazine er...|
|crossword puzzles...|
|run, blago run! s...|
|it might become a...|
+--------------------+
only showing top 20 rows

List(project_id, name, desc, goal, keywords, final_status, country2, currency2, days_campaign, hours_prepa, text)

import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions.concat_ws
colsToConcat: Seq[org.apache.spark.sql.Column] = List(name, desc, keywords)
dfCountry6: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]


#### On remplace les valeurs nulles des colonnes days_campaign, hours_prepa, et goal par la valeur -1 et par "unknown" pour les colonnes country2 et currency2 :

In [119]:
val dfCountry7 = dfCountry6
                        .na.fill("-1" , Seq("days_campaign"))
                        .na.fill("-1" , Seq("hours_prepa"))
                        .na.fill("-1" , Seq("goal"))


dfCountry7.select("days_campaign","hours_prepa","goal","country2","currency2").show()
print(dfCountry7.columns.toList)
dfCountry7
    .select("days_campaign", "hours_prepa","goal","country2","currency2")
    .describe()
    .show

+-------------+-----------+-----+--------+---------+
|days_campaign|hours_prepa| goal|country2|currency2|
+-------------+-----------+-----+--------+---------+
|        8.464|      0.616|   20|      US|      USD|
|       16.822|      4.269|  300|      US|      USD|
|         9.99|      0.218|   30|      US|      USD|
|       29.965|      0.815|  500|      US|      USD|
|       29.969|       0.73| 2000|      US|      USD|
|       27.212|    114.908|  700|      US|      USD|
|       24.719|      3.093|  250|      US|      USD|
|       30.298|      2.708| 1000|      US|      USD|
|       32.216|      18.26| 5000|      US|      USD|
|       30.938|      1.473| 3500|      US|      USD|
|       34.376|     62.977|30000|      US|      USD|
|       29.046|      1.596|  300|      US|      USD|
|       30.938|      1.488| 1500|      US|      USD|
|       46.909|      1.309|  500|      US|      USD|
|       36.977|     72.544|  500|      US|      USD|
|        33.18|       2.19| 1000|      US|    

dfCountry7: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]


In [ ]:
val monDataFrameFinal = dfCountry7
monDataFrameFinal.write.parquet("/mnt/d/09_SPARK/monDataFrameFinal")